In [1]:
import torch
import kaldi_io
import kaldiio
import json
import re
import numpy as np

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## 读取feats.scp

In [2]:
feats_map = {}
with open("/home1/meichaoyang/dataset/magic_aug/data_reverb/feats.scp", "r") as f:
    for line in f:
        data = line.split()
        feats_map[data[0]] = data[1]

In [3]:
feats_map["reverb1-14_3466_20170826171159"]

'/home1/meichaoyang/dataset/magic_aug/data_reverb/_fbank/raw_fbank_data_reverb.1.ark:31'

## 读取corpus.txt

In [4]:
corpus_map = {}
with open('/home1/meichaoyang/dataset/magic_aug/data_reverb/text_old', 'r') as f:
    for line in f:
        data = line.split()
        corpus_txt = re.sub(r"([.!?。！，？、 \[\],，])", r"", data[1])
        corpus_map[data[0]] = corpus_txt

In [5]:
SOS_token = 0
EOS_token = 1
import re
# string = '我要把你卸载掉'
# re.findall(r'.{1}', string)

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS":0, "EOS":1, "<unk>":2}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS",2: "<unk>"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in re.findall(r'.{1}', sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [6]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?。！，？、 \[\],，])", r" \1", s)
    return s

def readLangs(lang1):
    print("Reading lines...")
    lang = Lang(lang1)

    # Read the file and split into lines
    lines = open('/home1/meichaoyang/dataset/magic_aug/data_reverb/text_old', "r").\
        read().strip().split('\n')

    for line in lines:
        s = line.split()
        lang.addSentence(normalizeString(s[1]))

    return lang

In [7]:
lang = readLangs("ZH")

Reading lines...


# 准备json数据

In [8]:
json_data = {}
utts = json_data["utts"] = {}
dic = json_data["dic"] = {}
for i in range(lang.n_words):
    dic[lang.index2word[i]] = i
    lang.index2word[i]

In [34]:
# IC0001W0001
for ch in corpus_map["IC0001W0001"]:
    print(ch)

厨
房
用
具


In [9]:
keys = list(feats_map.keys())
for utt in keys:
    utts[utt] = {}
    utts[utt]["input"] = {}
    utts[utt]["input"]["feat"] = feats_map[utt]
    utts[utt]["input"]["shape"] = list(kaldi_io.read_mat(feats_map[utt]).shape)
    utts[utt]["output"] = {}
    utts[utt]["output"]["text"] = corpus_map[utt]
    tokenid = [lang.word2index[ch] if ch in lang.word2index else lang.word2index['<unk>']  for ch in corpus_map[utt]]
    utts[utt]["output"]["tokenid"] = tokenid
    utts[utt]["output"]["shape"] = [len(tokenid), lang.n_words]

In [26]:
a = kaldi_io.read_mat(feats_map["reverb1-14_3466_20170826171159"])

In [21]:
a[:a.shape[0]//4*4].shape
# a.shape[0]//4*4

(304, 80)

In [9]:
b = np.array([[1,2,3],[4,5,6]])

In [10]:
b[:,0:2]

array([[1, 2],
       [4, 5]])

In [ ]:
lang.word2index

In [21]:
with open("train_chars.txt" ,"w") as f:
    for i in lang.word2index:
        f.write(i+" "+str(lang.word2index[i])+"\n")

In [27]:
utts["reverb1-14_3466_20170826171159"]["output"]

{'text': '请语言播放小说', 'tokenid': [2, 3, 4, 5, 6, 7, 8], 'shape': [7, 4519]}

## 写入json文件

In [10]:
with open('data.json', 'w') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

## 读取json文件

In [20]:
with open('data.json', 'r') as f:
    json_data = json.load(f)

In [23]:
len(json_data["utts"])

573480

In [11]:
feats_map = {}
with open("/home1/meichaoyang/dataset/data_aishell2/feats/test/feats.scp", "r") as f:
    for line in f:
        data = line.split()
        feats_map[data[0]] = data[1]

In [12]:
corpus_map = {}
with open('/home1/meichaoyang/dataset/data_aishell2/feats/test/text', 'r') as f:
    for line in f:
        data = line.split()
        corpus_txt = re.sub(r"([.!?。！，？、 \[\],，])", r"", data[1])
        corpus_map[data[0]] = corpus_txt

In [13]:
json_data = {}
utts = json_data["utts"] = {}
dic = json_data["dic"] = {}
for i in range(lang.n_words):
    dic[lang.index2word[i]] = i
    lang.index2word[i]

In [14]:
keys = list(feats_map.keys())
for utt in keys:
    utts[utt] = {}
    utts[utt]["input"] = {}
    utts[utt]["input"]["feat"] = feats_map[utt]
    utts[utt]["input"]["shape"] = list(kaldi_io.read_mat(feats_map[utt]).shape)
    utts[utt]["output"] = {}
    utts[utt]["output"]["text"] = corpus_map[utt]
    tokenid = [lang.word2index[ch] if ch in lang.word2index else lang.word2index['<unk>']  for ch in corpus_map[utt]]
    utts[utt]["output"]["tokenid"] = tokenid
    utts[utt]["output"]["shape"] = [len(tokenid), lang.n_words]

In [15]:
with open('data_test.json', 'w') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

### 测试data4.py

In [16]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import sys
import gc
import json
import time
from data_4 import AudioDataLoader, AudioDataset, pad_list

%matplotlib inline

print_use = False

In [17]:
train_json = "data.json"
test_json = "data_test.json"
batch_size = 32
maxlen_in = 100000
maxlen_out = 30
num_workers = 4

In [18]:
te_dataset = AudioDataset(test_json, batch_size,
                              maxlen_in, maxlen_out)
te_loader = AudioDataLoader(te_dataset, batch_size=1, num_workers=num_workers)

In [19]:
for i, (data) in enumerate(te_loader):
    print(str(i)+" : "+str(data))
    break

0 : (tensor([[[ 4.8704,  7.6203, 10.4137,  ..., 13.1590, 13.5756, 13.2265],
         [ 4.9864,  7.2130,  8.9778,  ..., 13.0554, 13.8832, 13.1171],
         [ 3.2858,  6.8057,  9.6691,  ..., 12.8481, 12.9974, 12.5761],
         ...,
         [ 0.9360,  5.2082,  6.6501,  ..., 12.3645, 11.8783, 12.0713],
         [ 1.5080,  3.2681,  5.2221,  ..., 12.3645, 11.9866, 11.5665],
         [ 4.1747,  5.6292,  5.7219,  ..., 12.4336, 11.9866, 11.5350]],

        [[ 1.1541,  4.0020,  4.9308,  ..., 12.2372, 12.0145, 11.6568],
         [ 1.6574,  4.6347,  5.9078,  ..., 12.0907, 12.0145, 11.5204],
         [ 3.7213,  5.0948,  6.1032,  ..., 11.8814, 11.9975, 11.7713],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 3.3843,  6.3341,  7.3461,  ..., 11.7156, 12.2395, 11.6748],
         [ 3.1986,  6.3341,  7.1736,  ..

In [4]:
for i, (data) in enumerate(te_loader):
    print(str(i)+" : "+str(data))
    break

32
32


32
----------------------------------


-----------------2


2

2
32
32



----------------------------------
2

2


32

-----------------
2

0 : None


In [4]:
for i, (data) in enumerate(te_loader):
    print(str(i)+" : "+str(data))
    break

('IC0085W0306', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.15.ark:193509', 'shape': [926, 40]}, 'output': {'text': '并成功实现了商流信息流资金流与物流的四流合一', 'tokenid': [1073, 309, 1100, 1207, 136, 63, 1510, 149, 369, 997, 149, 1563, 497, 149, 501, 132, 149, 20, 366, 149, 45, 66], 'shape': [22, 4519]}})('IC0094W0460', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.24.ark:36078', 'shape': [1166, 40]}, 'output': {'text': '王辰澎湃资料八月十八日下午二点二十分左右在运行时梯街突然跳起', 'tokenid': [595, 1150, 2890, 3877, 1563, 1564, 777, 745, 27, 777, 521, 142, 966, 760, 74, 760, 27, 421, 1824, 2063, 51, 625, 137, 83, 1309, 127, 789, 408, 1635, 404], 'shape': [30, 4519]}})

('IC0086W0431', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.16.ark:1905953', 'shape': [814, 40]}, 'output': {'text': '上校阅兵后获批率直升机绕飞仙桃市向家乡致意', 'tokenid': [54, 516, 1360, 586, 114, 1963, 1943, 1593, 1041, 403, 544, 2440, 40

In [4]:
for i, (data) in enumerate(te_loader):
    print(str(i)+" : "+str(data))
    break

[('IC0096W0301', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.25.ark:5912144', 'shape': [1325, 40]}, 'output': {'text': '就是加入了P点对点对等网络传输的更新方式', 'tokenid': [220, 60, 285, 346, 63, '<unk>', 74, 935, 74, 935, 537, 651, 652, 664, 1746, 20, 557, 184, 292, 821], 'shape': [20, 4519]}}), ('IC0094W0460', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.24.ark:36078', 'shape': [1166, 40]}, 'output': {'text': '王辰澎湃资料八月十八日下午二点二十分左右在运行时梯街突然跳起', 'tokenid': [595, 1150, 2890, 3877, 1563, 1564, 777, 745, 27, 777, 521, 142, 966, 760, 74, 760, 27, 421, 1824, 2063, 51, 625, 137, 83, 1309, 127, 789, 408, 1635, 404], 'shape': [30, 4519]}}), ('ID0042W0471', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.41.ark:6090243', 'shape': [1146, 40]}, 'output': {'text': '但作为深圳国资委旗下最大的房产房地产但作为深圳国资委旗下最大的房地产公司', 'tokenid': [552, 547, 200, 695, 1193, 468, 1563, 1721, 1890, 142, 208, 

[('IC0007W0264', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:3152908', 'shape': [927, 40]}, 'output': {'text': 'CX六百采用的也并非是FIREFOX操作系统', 'tokenid': ['<unk>', '<unk>', 283, 988, 2394, 261, 20, 229, 1073, 959, 60, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 536, 547, 1009, 1072], 'shape': [22, 4519]}}), ('IC0085W0306', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.15.ark:193509', 'shape': [926, 40]}, 'output': {'text': '并成功实现了商流信息流资金流与物流的四流合一', 'tokenid': [1073, 309, 1100, 1207, 136, 63, 1510, 149, 369, 997, 149, 1563, 497, 149, 501, 132, 149, 20, 366, 149, 45, 66], 'shape': [22, 4519]}}), ('IC0082W0021', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.11.ark:5199439', 'shape': [924, 40]}, 'output': {'text': '幺九四九THEHIDDENCCITY三里屯店', 'tokenid': [1844, 419, 366, 419, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<

[('ID0041W0413', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.40.ark:5453061', 'shape': [849, 40]}, 'output': {'text': '中海集运二零一一年全年净污损达七亿元', 'tokenid': [347, 448, 725, 625, 760, 783, 66, 66, 303, 11, 303, 582, 2658, 1431, 752, 748, 2721, 1323], 'shape': [18, 4519]}}), ('IC0007W0429', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:6598833', 'shape': [848, 40]}, 'output': {'text': '有天坐巴士去赤柱以及去愉景湾参加朋友婚礼', 'tokenid': [107, 78, 128, 763, 788, 125, 533, 3101, 48, 435, 125, 1464, 266, 1917, 1326, 285, 477, 478, 838, 1411], 'shape': [20, 4519]}}), ('IC0091W0496', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.21.ark:1598749', 'shape': [848, 40]}, 'output': {'text': '上海国际和平妇幼保健院急诊科一名医生遭女患者殴打', 'tokenid': [54, 448, 468, 1166, 232, 1223, 300, 2204, 628, 563, 515, 455, 1677, 1523, 66, 444, 841, 25, 2600, 766, 2977, 281, 4393, 94], 'shape': [24, 4519]}

[('IC0085W0214', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.14.ark:7183574', 'shape': [879, 40]}, 'output': {'text': '某保险公司权益投资部总经理曾某借职务之便建老鼠仓', 'tokenid': [71, 628, 629, 172, 929, 1480, 2279, 49, 1563, 1081, 878, 367, 666, 1935, 71, 714, 2124, 1018, 339, 293, 917, 282, 1441, 2852], 'shape': [24, 4519]}}), ('IC0009W0360', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.9.ark:3432159', 'shape': [878, 40]}, 'output': {'text': '首先由带有浓浓山海关特色的项目一古城定向开场', 'tokenid': [207, 564, 1092, 494, 107, 2702, 2702, 523, 448, 179, 336, 724, 20, 2447, 723, 66, 1128, 480, 87, 543, 95, 545], 'shape': [22, 4519]}}), ('IC0007W0129', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:297573', 'shape': [877, 40]}, 'output': {'text': '持股比例直逼宁波银行第一大股东新加坡华侨银行', 'tokenid': [1485, 950, 268, 454, 1041, 2357, 1069, 1735, 659, 137, 254, 66, 217, 950, 359, 184, 285, 1853, 562

0 : None


In [10]:
len(d)

32

In [6]:
for i, (data) in enumerate(te_loader):
    print(i+" : "+data)
    break

[[('IC0096W0301', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.25.ark:5912144', 'shape': [1325, 40]}, 'output': {'text': '就是加入了P点对点对等网络传输的更新方式', 'tokenid': [220, 60, 285, 346, 63, '<unk>', 74, 935, 74, 935, 537, 651, 652, 664, 1746, 20, 557, 184, 292, 821], 'shape': [20, 4519]}}), ('IC0094W0460', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.24.ark:36078', 'shape': [1166, 40]}, 'output': {'text': '王辰澎湃资料八月十八日下午二点二十分左右在运行时梯街突然跳起', 'tokenid': [595, 1150, 2890, 3877, 1563, 1564, 777, 745, 27, 777, 521, 142, 966, 760, 74, 760, 27, 421, 1824, 2063, 51, 625, 137, 83, 1309, 127, 789, 408, 1635, 404], 'shape': [30, 4519]}}), ('ID0042W0471', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.41.ark:6090243', 'shape': [1146, 40]}, 'output': {'text': '但作为深圳国资委旗下最大的房产房地产但作为深圳国资委旗下最大的房地产公司', 'tokenid': [552, 547, 200, 695, 1193, 468, 1563, 1721, 1890, 142, 208,

[[('IC0007W0264', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:3152908', 'shape': [927, 40]}, 'output': {'text': 'CX六百采用的也并非是FIREFOX操作系统', 'tokenid': ['<unk>', '<unk>', 283, 988, 2394, 261, 20, 229, 1073, 959, 60, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 536, 547, 1009, 1072], 'shape': [22, 4519]}}), ('IC0085W0306', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.15.ark:193509', 'shape': [926, 40]}, 'output': {'text': '并成功实现了商流信息流资金流与物流的四流合一', 'tokenid': [1073, 309, 1100, 1207, 136, 63, 1510, 149, 369, 997, 149, 1563, 497, 149, 501, 132, 149, 20, 366, 149, 45, 66], 'shape': [22, 4519]}}), ('IC0082W0021', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.11.ark:5199439', 'shape': [924, 40]}, 'output': {'text': '幺九四九THEHIDDENCCITY三里屯店', 'tokenid': [1844, 419, 366, 419, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '

[[('ID0041W0413', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.40.ark:5453061', 'shape': [849, 40]}, 'output': {'text': '中海集运二零一一年全年净污损达七亿元', 'tokenid': [347, 448, 725, 625, 760, 783, 66, 66, 303, 11, 303, 582, 2658, 1431, 752, 748, 2721, 1323], 'shape': [18, 4519]}}), ('IC0007W0429', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:6598833', 'shape': [848, 40]}, 'output': {'text': '有天坐巴士去赤柱以及去愉景湾参加朋友婚礼', 'tokenid': [107, 78, 128, 763, 788, 125, 533, 3101, 48, 435, 125, 1464, 266, 1917, 1326, 285, 477, 478, 838, 1411], 'shape': [20, 4519]}}), ('IC0091W0496', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.21.ark:1598749', 'shape': [848, 40]}, 'output': {'text': '上海国际和平妇幼保健院急诊科一名医生遭女患者殴打', 'tokenid': [54, 448, 468, 1166, 232, 1223, 300, 2204, 628, 563, 515, 455, 1677, 1523, 66, 444, 841, 25, 2600, 766, 2977, 281, 4393, 94], 'shape': [24, 4519]

----------------------------------

-----------------


[('IC0007W0264', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:3152908', 'shape': [927, 40]}, 'output': {'text': 'CX六百采用的也并非是FIREFOX操作系统', 'tokenid': ['<unk>', '<unk>', 283, 988, 2394, 261, 20, 229, 1073, 959, 60, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 536, 547, 1009, 1072], 'shape': [22, 4519]}}), ('IC0085W0306', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.15.ark:193509', 'shape': [926, 40]}, 'output': {'text': '并成功实现了商流信息流资金流与物流的四流合一', 'tokenid': [1073, 309, 1100, 1207, 136, 63, 1510, 149, 369, 997, 149, 1563, 497, 149, 501, 132, 149, 20, 366, 149, 45, 66], 'shape': [22, 4519]}}), ('IC0082W0021', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.11.ark:5199439', 'shape': [924, 40]}, 'output': {'text': '幺九四九THEHIDDENCCITY三里屯店', 'tokenid': [1844, 419, 366, 419, '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<

...

[('IC0096W0217', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.25.ark:4231612', 'shape': [830, 40]}, 'output': {'text': '问这次审计中的新农合资金是由卫生部管理的', 'tokenid': [77, 9, 1204, 2052, 1305, 347, 20, 184, 1065, 45, 1563, 497, 60, 1092, 823, 25, 1081, 897, 666, 20], 'shape': [20, 4519]}}), ('ID0023W0422', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.32.ark:4555552', 'shape': [830, 40]}, 'output': {'text': '位于冕宁县灵山景区有七名宿营大学生因突发暴雨被困', 'tokenid': [57, 1398, 2973, 1069, 307, 1648, 523, 266, 396, 107, 748, 444, 1283, 1942, 217, 238, 25, 377, 789, 570, 513, 218, 392, 1468], 'shape': [24, 4519]}}), ('IC0002W0212', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.2.ark:2972068', 'shape': [828, 40]}, 'output': {'text': '这种模式不能保证所采购产品是市场上最好的产品', 'tokenid': [9, 568, 820, 821, 42, 177, 628, 859, 434, 2394, 111, 1300, 490, 60, 22, 545, 54, 208, 120, 20, 1300, 490], 'sha

[('ID0041W0505', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.40.ark:7505017', 'shape': [815, 40]}, 'output': {'text': '江苏重复地名让人傻傻分不清三个周庄两个茅山', 'tokenid': [356, 374, 573, 277, 52, 444, 385, 72, 1327, 1327, 421, 42, 768, 24, 39, 227, 450, 505, 39, 3452, 523], 'shape': [21, 4519]}}), ('IC0086W0431', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.16.ark:1905953', 'shape': [814, 40]}, 'output': {'text': '上校阅兵后获批率直升机绕飞仙桃市向家乡致意', 'tokenid': [54, 516, 1360, 586, 114, 1963, 1943, 1593, 1041, 403, 544, 2440, 406, 1781, 28, 22, 543, 417, 495, 1054, 439], 'shape': [21, 4519]}}), ('IC0008W0306', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.8.ark:3329467', 'shape': [813, 40]}, 'output': {'text': '并于一零一二年推出FACE加加人脸识别云平台', 'tokenid': [1073, 1398, 66, 783, 66, 760, 303, 64, 56, '<unk>', '<unk>', '<unk>', '<unk>', 285, 285, 72, 1010, 1450, 279, 487, 1223, 67],

[('IC0085W0214', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.14.ark:7183574', 'shape': [879, 40]}, 'output': {'text': '某保险公司权益投资部总经理曾某借职务之便建老鼠仓', 'tokenid': [71, 628, 629, 172, 929, 1480, 2279, 49, 1563, 1081, 878, 367, 666, 1935, 71, 714, 2124, 1018, 339, 293, 917, 282, 1441, 2852], 'shape': [24, 4519]}}), ('IC0009W0360', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.9.ark:3432159', 'shape': [878, 40]}, 'output': {'text': '首先由带有浓浓山海关特色的项目一古城定向开场', 'tokenid': [207, 564, 1092, 494, 107, 2702, 2702, 523, 448, 179, 336, 724, 20, 2447, 723, 66, 1128, 480, 87, 543, 95, 545], 'shape': [22, 4519]}}), ('IC0007W0129', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.7.ark:297573', 'shape': [877, 40]}, 'output': {'text': '持股比例直逼宁波银行第一大股东新加坡华侨银行', 'tokenid': [1485, 950, 268, 454, 1041, 2357, 1069, 1735, 659, 137, 254, 66, 217, 950, 359, 184, 285, 1853, 562




('IC0096W0217', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.25.ark:4231612', 'shape': [830, 40]}, 'output': {'text': '问这次审计中的新农合资金是由卫生部管理的', 'tokenid': [77, 9, 1204, 2052, 1305, 347, 20, 184, 1065, 45, 1563, 497, 60, 1092, 823, 25, 1081, 897, 666, 20], 'shape': [20, 4519]}})-----------------
('ID0041W0505', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.40.ark:7505017', 'shape': [815, 40]}, 'output': {'text': '江苏重复地名让人傻傻分不清三个周庄两个茅山', 'tokenid': [356, 374, 573, 277, 52, 444, 385, 72, 1327, 1327, 421, 42, 768, 24, 39, 227, 450, 505, 39, 3452, 523], 'shape': [21, 4519]}})

('IC0085W0214', {'input': {'feat': '/home1/meichaoyang/dataset/data_aishell2/feats/test/_fbank/raw_fbank_test.14.ark:7183574', 'shape': [879, 40]}, 'output': {'text': '某保险公司权益投资部总经理曾某借职务之便建老鼠仓', 'tokenid': [71, 628, 629, 172, 929, 1480, 2279, 49, 1563, 1081, 878, 367, 666, 1935, 71, 714, 2124, 1018, 339, 293, 917, 282, 14

TypeError: unsupported operand type(s) for +: 'int' and 'str'